In [1]:
import torch
import matplotlib.pyplot as plt
import sys
from neuralop.models import FNO, F_FNO2D

from neuralop import Trainer
from neuralop.training import OutputEncoderCallback
from neuralop.utils import count_params
from neuralop import LpLoss, H1Loss

device = 'cuda'

from neuralop.datasets.autoregressive_dataset import load_autoregressive_traintestsplit
data_path = "./data/NavierStokes_V1e-5_N1200_T20.mat"

n_train = 1000
n_test = 200
batch_size = 32
test_batch_size = 64
train_subsample_rate = 1
test_subsample_rate = 1
time_step = 1
train_loader, test_loader = load_autoregressive_traintestsplit(
    data_path,
    n_train, n_test,
    batch_size, test_batch_size, 
    train_subsample_rate, test_subsample_rate,
    time_step,
    predict_feature='u',
)

/home/yichen/anaconda3/envs/test/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/yichen/anaconda3/envs/test/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda9SetDeviceEi'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/yichen/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "FFNO"

In [3]:
# We create a F-FNO model

input_prods = None
n_modes=21

model = F_FNO2D(in_channels=3, n_modes=(n_modes, n_modes), hidden_channels=32, 
             projection_channels=64, factorization='tucker', channel_mixing='prod-layer', ffno_channel_mixing='prod', stabilizer='tanh', rank=0.42)

# model = FNO(in_channels=3, n_modes=(n_modes, n_modes), hidden_channels=32, 
#              projection_channels=64, factorization='tucker', channel_mixing='prod-layer', stabilizer='tanh', rank=0.42)

# method_name = type(model).__name__.split('.')[-1]

model = model.to(device)

n_params = count_params(model)
print(f'\nOur '+model_name+f' model has {n_params} parameters.')

sys.stdout.flush()


Our FFNO model has 186369 parameters.


In [4]:
#Create the optimizer
optimizer = torch.optim.Adam(model.parameters(), 
                                lr=1e-3, 
                                weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)

# Creating the losses
l2loss = LpLoss(d=2, p=2)
h1loss = H1Loss(d=2)

train_loss = h1loss
eval_losses={'h1': h1loss, 'l2': l2loss}

print('\n### MODEL ###\n', model)
print('\n### OPTIMIZER ###\n', optimizer)
print('\n### SCHEDULER ###\n', scheduler)
print('\n### LOSSES ###')
print(f'\n * Train: {train_loss}')
print(f'\n * Test: {eval_losses}')
sys.stdout.flush()


### MODEL ###
 F_FNO2D(
  (fno_blocks): F_FNOBlocks2D(
    (fno_skips): ModuleList(
      (0-3): 4 x Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
    (channel_mixer): ModuleList(
      (0-3): 4 x ProductLayer(
        (linear): MLP(
          (fcs): ModuleList(
            (0): Conv2d(34, 32, kernel_size=(1, 1), stride=(1, 1))
          )
        )
      )
    )
    (mixer_skips): ModuleList(
      (0-3): 4 x SoftGating()
    )
    (convs): SpectralConvFFNO2d(
      (weight): ModuleList(
        (0-7): 8 x ComplexDenseTensor(shape=torch.Size([32, 32, 10]), rank=None)
      )
      (linear): MLP(
        (fcs): ModuleList(
          (0): Conv2d(66, 32, kernel_size=(1, 1), stride=(1, 1))
        )
      )
    )
  )
  (lifting): MLP(
    (fcs): ModuleList(
      (0): Conv2d(3, 256, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
    )
  )
  (projection): MLP(
    (fcs): ModuleList(
      (0): Conv2d(32, 64, kernel_si

In [5]:
import time
localtime = time.localtime(time.time())
time_now = f"{localtime.tm_mon}-{localtime.tm_mday}-{localtime.tm_hour}-{localtime.tm_min}"

In [6]:
from neuralop.training import MultipleInputCallback, SimpleTensorBoardLoggerCallback
trainer = Trainer(model=model, n_epochs=500,
                  device=device,
                  callbacks=[MultipleInputCallback(append_positional_encoding=True), SimpleTensorBoardLoggerCallback(log_dir='runs/TorisLi_exp_'+model_name+time_now,)],             
                  wandb_log=False,
                  log_test_interval=4,
                  use_distributed=False,
                  verbose=True)

2024-04-26 20:14:00.770027: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-26 20:14:00.910943: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-26 20:14:00.911008: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-26 20:14:00.932808: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-26 20:14:00.976756: I tensorflow/core/platform/cpu_feature_guar

using standard method to load data to device.
using standard method to compute loss.
self.override_load_to_device=False
self.overrides_loss=False


In [7]:
trainer.train(train_loader=train_loader,
              test_loaders={64: test_loader},
              optimizer=optimizer, 
              scheduler=scheduler, 
              regularizer=False, 
              training_loss=train_loss, 
              eval_losses=eval_losses)

Training on 19000 samples
Testing on [3800] samples         on resolutions [64].
Raw outputs of size out.shape=torch.Size([32, 1, 64, 64])
[0] time=11.05, avg_loss=0.0149, train_err=0.0004, 64_h1=0.3423, 64_l2=0.1348
[4] time=9.86, avg_loss=0.0107, train_err=0.0003, 64_h1=0.2353, 64_l2=0.0833


KeyboardInterrupt: 

In [10]:
torch.save(trainer.model.state_dict(), "./ckpt/TorisLi/"+model_name+"_ep500.pth")

In [14]:
test_samples = test_loader.dataset

fig = plt.figure(figsize=(7, 7))
for index in range(3):
    data = test_samples[index]
    # Input x
    x = data['x']
    # Ground-truth
    y = data['y']
    # Model prediction
    out = model(x.unsqueeze(0))

    ax = fig.add_subplot(3, 3, index*3 + 1)
    ax.imshow(x[0], cmap='gray')
    if index == 0: 
        ax.set_title('Input x')
    plt.xticks([], [])
    plt.yticks([], [])

    ax = fig.add_subplot(3, 3, index*3 + 2)
    ax.imshow(y.squeeze())
    if index == 0: 
        ax.set_title('Ground-truth y')
    plt.xticks([], [])
    plt.yticks([], [])

    ax = fig.add_subplot(3, 3, index*3 + 3)
    ax.imshow(out.squeeze().detach().numpy())
    if index == 0: 
        ax.set_title('Model prediction')
    plt.xticks([], [])
    plt.yticks([], [])

fig.suptitle('Inputs, ground-truth output and prediction.', y=0.98)
plt.tight_layout()
fig.show()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/yichen/anaconda3/envs/test/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_44388/849268122.py", line 11, in <module>
    out = model(x.unsqueeze(0))
  File "/home/yichen/anaconda3/envs/test/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/yichen/repo/cfd/myNeuralOperator/neuralop/models/new_fno.py", line 252, in forward
  File "/home/yichen/anaconda3/envs/test/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/yichen/repo/cfd/myNeuralOperator/neuralop/layers/mlp.py", line 62, in forward
    x = fc(x)
  File "/home/yichen/anaconda3/envs/test/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args

<Figure size 700x700 with 0 Axes>